测试链接表格，首先检查表格内容

In [16]:
import pandas as pd

# 1. 迁移流
mig = pd.read_csv('../data/processed/migration_flow_cleaned.csv')
print("迁移流表 columns:", mig.columns.tolist())
print(mig.head())

# 2. GDP
gdp = pd.read_csv('../data/processed/gdp_country_year_2019_2022_cleaned.csv')
print("GDP表 columns:", gdp.columns.tolist())
print(gdp.head())

# 3. Market Access
ma = pd.read_csv('../data/processed/market_access_panel.csv')
print("Market Access表 columns:", ma.columns.tolist())
print(ma.head())

# 4. SPEI
spei = pd.read_csv('../data/processed/spei03_country_month_cleaned.csv')
print("SPEI表 columns:", spei.columns.tolist())
print(spei.head())

# 5. 边境摩擦
border = pd.read_csv('../data/processed/border_friction_panel_ew.csv')
print("边境摩擦表 columns:", border.columns.tolist())
print(border.head())

迁移流表 columns: ['origin_iso3', 'destination_iso3', 'migration_month', 'year', 'month', 'flow', 'log_flow', 'origin_iso2', 'destination_iso2']
  origin_iso3 destination_iso3 migration_month  year  month  flow  log_flow  \
0         AND              ARE      2019-01-01  2019      1    12  2.564949   
1         AND              ARE      2019-02-01  2019      2     2  1.098612   
2         AND              ARE      2019-03-01  2019      3     1  0.693147   
3         AND              ARE      2019-04-01  2019      4     7  2.079442   
4         AND              ARE      2019-05-01  2019      5     0  0.000000   

  origin_iso2 destination_iso2  
0          AD               AE  
1          AD               AE  
2          AD               AE  
3          AD               AE  
4          AD               AE  
GDP表 columns: ['iso3', 'year', 'gdp']
  iso3  year           gdp
0  AFE  2019  1.009747e+12
1  AFW  2019  8.332889e+11
2  ARB  2019  2.949355e+12
3  CSS  2019  6.192930e+10
4  CEB  2019 

In [17]:
# 检查各表主键列、数据类型、缺失值等，为合并做准备

def check_merge_ready():
    print("=== 迁移流表 ===")
    show_info(mig, "迁移流表")
    print("\n主键示例:", mig[['origin_iso3', 'destination_iso3', 'year', 'month']].head())

    print("\n=== GDP表 ===")
    show_info(gdp, "GDP表")
    print("\n主键示例:", gdp[['iso3', 'year']].head())

    print("\n=== Market Access表 ===")
    show_info(ma, "Market Access表")
    print("\n主键示例:", ma[['iso3', 'year']].head())

    print("\n=== SPEI表 ===")
    show_info(spei, "SPEI表")
    print("\n主键示例:", spei[['origin', 'year', 'month']].head())

    print("\n=== 边境摩擦表 ===")
    show_info(border, "边境摩擦表")
    print("\n主键示例:", border[['iso_o', 'iso_d']].head())

    # 检查主键列类型
    print("\n迁移流表 origin_iso3 类型:", mig['origin_iso3'].dtype)
    print("GDP表 iso3 类型:", gdp['iso3'].dtype)
    print("Market Access表 iso3 类型:", ma['iso3'].dtype)
    print("SPEI表 origin 类型:", spei['origin'].dtype)
    print("边境摩擦表 iso_o 类型:", border['iso_o'].dtype)

    # 检查主键列是否有缺失
    print("\n迁移流表 origin_iso3 缺失:", mig['origin_iso3'].isnull().sum())
    print("GDP表 iso3 缺失:", gdp['iso3'].isnull().sum())
    print("Market Access表 iso3 缺失:", ma['iso3'].isnull().sum())
    print("SPEI表 origin 缺失:", spei['origin'].isnull().sum())
    print("边境摩擦表 iso_o 缺失:", border['iso_o'].isnull().sum())

check_merge_ready()

=== 迁移流表 ===

迁移流表  shape: (1545874, 9)
迁移流表 columns: ['origin_iso3', 'destination_iso3', 'migration_month', 'year', 'month', 'flow', 'log_flow', 'origin_iso2', 'destination_iso2']
迁移流表 缺失值统计:
origin_iso3         0
destination_iso3    0
migration_month     0
year                0
month               0
flow                0
log_flow            0
origin_iso2         0
destination_iso2    0
dtype: int64
迁移流表 数据类型:
origin_iso3          object
destination_iso3     object
migration_month      object
year                  int64
month                 int64
flow                  int64
log_flow            float64
origin_iso2          object
destination_iso2     object
dtype: object
迁移流表 基本统计:
       origin_iso3 destination_iso3 migration_month          year  \
count      1545874          1545874         1545874  1.545874e+06   
unique         180              180              48           NaN   
top            AND              ARE      2019-04-01           NaN   
freq          8592             8

KeyError: "None of [Index(['origin', 'year', 'month'], dtype='object')] are in the [columns]"

In [7]:
def show_info(df, name):
    print(f"\n{name}  shape: {df.shape}")
    print(f"{name} columns: {df.columns.tolist()}")
    print(f"{name} 缺失值统计:\n{df.isnull().sum()}")
    print(f"{name} 数据类型:\n{df.dtypes}")
    print(f"{name} 基本统计:\n{df.describe(include='all')}")

show_info(mig, "迁移流表（merge前）")


迁移流表（merge前）  shape: (1545874, 9)
迁移流表（merge前） columns: ['origin_iso3', 'destination_iso3', 'migration_month', 'year', 'month', 'flow', 'log_flow', 'origin', 'destination']
迁移流表（merge前） 缺失值统计:
origin_iso3         0
destination_iso3    0
migration_month     0
year                0
month               0
flow                0
log_flow            0
origin              0
destination         0
dtype: int64
迁移流表（merge前） 数据类型:
origin_iso3          object
destination_iso3     object
migration_month      object
year                  int64
month                 int64
flow                  int64
log_flow            float64
origin               object
destination          object
dtype: object
迁移流表（merge前） 基本统计:
       origin_iso3 destination_iso3 migration_month          year  \
count      1545874          1545874         1545874  1.545874e+06   
unique         180              180              48           NaN   
top            AND              ARE      2019-04-01           NaN   
freq          8

In [14]:
# 1. 处理 SPEI 日期，提取 year 和 month
spei['date'] = pd.to_datetime(spei['date'])
spei['year'] = spei['date'].dt.year
spei['month'] = spei['date'].dt.month

# 2. 重命名 ISO_A3 为 origin，spei 为 spei_origin
spei = spei.rename(columns={'ISO_A3': 'origin', 'spei': 'spei_origin'})

# 3. 合并
mig_spei = mig.merge(
    spei[['origin', 'year', 'month', 'spei_origin']],
    left_on=['origin_iso3', 'year', 'month'],
    right_on=['origin', 'year', 'month'],
    how='left'
)


In [15]:
# 4. 检查合并结果
print(mig_spei.head())
print("缺失SPEI的行数：", mig_spei['spei_origin'].isnull().sum())
print(mig_spei.shape)
print(mig_spei.head())
print(mig_spei.isnull().sum())
print(mig_spei.dtypes)
print(mig_spei['spei_origin'].value_counts(dropna=False))

  origin_iso3 destination_iso3 migration_month  year  month  flow  log_flow  \
0         AND              ARE      2019-01-01  2019      1    12  2.564949   
1         AND              ARE      2019-02-01  2019      2     2  1.098612   
2         AND              ARE      2019-03-01  2019      3     1  0.693147   
3         AND              ARE      2019-04-01  2019      4     7  2.079442   
4         AND              ARE      2019-05-01  2019      5     0  0.000000   

  origin_x destination origin_y  spei_origin  
0       AD          AE      NaN          NaN  
1       AD          AE      NaN          NaN  
2       AD          AE      NaN          NaN  
3       AD          AE      NaN          NaN  
4       AD          AE      NaN          NaN  
缺失SPEI的行数： 146016
(1545874, 11)
  origin_iso3 destination_iso3 migration_month  year  month  flow  log_flow  \
0         AND              ARE      2019-01-01  2019      1    12  2.564949   
1         AND              ARE      2019-02-01  2019 